In [334]:
import pandas as pd
import numpy as np
import sqlalchemy
import pymysql
import re
from sqlalchemy import create_engine
from datetime import datetime
import sys

driver = 'mysql+pymysql'
ip = '127.0.0.1'
username = 'root'
password = ''
db = 'test'

cs  = f'{driver}://{username}:{password}@{ip}/{db}'
engine = create_engine(cs)
Ssql = 'SELECT * FROM st_bicing '


In [7]:
stat = pd.read_sql(Ssql,engine)


# Drop columns not needed 

In [14]:
stat.drop('streetName', axis=1, inplace = True)
stat.drop('streetNumber', axis=1, inplace = True)
stat.drop('nearbyStations', axis=1, inplace = True)
stat.drop('updateTime', axis=1, inplace = True)
stat.drop('month', axis=1, inplace = True)
stat.head()

,id,type,latitude,longitude,altitude,slots,bikes,status,day,time
0,1,BIKE,41.397952,2.180042,21.0,10,16,OPN,1,00:05:00
1,2,BIKE,41.395530,2.177060,21.0,8,18,OPN,1,00:05:00
2,3,BIKE,41.394055,2.181299,22.0,2,22,OPN,1,00:05:00
3,4,BIKE,41.393480,2.181555,21.0,0,21,OPN,1,00:05:00
4,5,BIKE,41.391075,2.180223,16.0,0,39,OPN,1,00:05:00


# Clean data

In [16]:
stat.info()
#Columns has ben casted using MySQL, from original column named 'updateTime' we have separated 
#into different columns with the day, time, etc. to manipulate operations within specific intervals of time

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3979843 entries, 0 to 3979842
Data columns (total 10 columns):
id           int64
type         object
latitude     float64
longitude    float64
altitude     float64
slots        int64
bikes        int64
status       object
day          int64
time         timedelta64[ns]
dtypes: float64(3), int64(4), object(2), timedelta64[ns](1)
memory usage: 303.6+ MB


In [297]:
Ssql1 = 'Select * from st_bicing where id = 1'
id_1 = pd.read_sql(Ssql1,engine)
id_1.drop('streetName', axis=1, inplace = True)
id_1.drop('streetNumber', axis=1, inplace = True)
id_1.drop('nearbyStations', axis=1, inplace = True)
id_1.drop('updateTime', axis=1, inplace = True)
id_1.drop('month', axis=1, inplace = True)
id_1.head()

,id,type,latitude,longitude,altitude,slots,bikes,status,day,time
0,1,BIKE,41.397952,2.180042,21.0,10,16,OPN,1,00:05:00
1,1,BIKE,41.397952,2.180042,21.0,10,16,OPN,1,00:09:00
2,1,BIKE,41.397952,2.180042,21.0,10,16,OPN,1,00:14:00
3,1,BIKE,41.397952,2.180042,21.0,9,17,OPN,1,00:20:00
4,1,BIKE,41.397952,2.180042,21.0,8,18,OPN,1,00:24:00


In [188]:
l_bikes = []
id_1['diff'] = id_1['bikes'].diff()
id_1.head()


,id,type,latitude,longitude,altitude,slots,bikes,status,day,time,diff
0,1,BIKE,41.397952,2.180042,21.0,10,16,OPN,1,00:05:00,NaN
1,1,BIKE,41.397952,2.180042,21.0,10,16,OPN,1,00:09:00,0.0
2,1,BIKE,41.397952,2.180042,21.0,10,16,OPN,1,00:14:00,0.0
3,1,BIKE,41.397952,2.180042,21.0,9,17,OPN,1,00:20:00,1.0
4,1,BIKE,41.397952,2.180042,21.0,8,18,OPN,1,00:24:00,1.0


In [251]:
#id_1_day = id_1[id_1['day']== 2]  
#id_1_day.head()

,id,type,latitude,longitude,altitude,slots,bikes,status,day,time,diff
265,1,BIKE,41.397952,2.180042,21.0,1,26,OPN,2,00:05:00,-1.0
266,1,BIKE,41.397952,2.180042,21.0,1,26,OPN,2,00:09:00,0.0
267,1,BIKE,41.397952,2.180042,21.0,1,26,OPN,2,00:15:00,0.0
268,1,BIKE,41.397952,2.180042,21.0,1,26,OPN,2,00:20:00,0.0
269,1,BIKE,41.397952,2.180042,21.0,1,26,OPN,2,00:24:00,0.0


In [252]:
#looking for the  max number of bikes taken in a moment
critical_outs = id_1_day.min()['diff']
critical_outs

-3.0

In [294]:
#looking for the index of the row
id = id_1_day['diff'][id_1_day['diff']== critical_outs].index[0]

#looking for the MOMENT where this happens (accessing by index)
a = id_1_day.loc[id,'time']

Timedelta('0 days 06:45:00')

In [296]:
#formatting the MOMENT as a STRING
time = re.findall(r'\d+:\d+:\d+',str(a))[0]
time

'06:45:00'

# Test with one station 

In [322]:
critical_outs_times = []
massive_ins_times = []


for day in range(1,32):
    id_1_day = id_1[id_1['day']== day]    #dataFrame of day 'day' 
    id_1_day['diff'] = id_1_day['bikes'].diff()

    #looking for the  max/min number of bikes taken in a moment
    critical_outs = id_1_day.min()['diff']
    massive_ins = id_1_day.max()['diff']

    #looking for the index of the rows
    id_out = id_1_day['diff'][id_1_day['diff']== critical_outs].index[0]
    id_in = id_1_day['diff'][id_1_day['diff']== massive_ins].index[0]

    #looking for the MOMENT where this happens (accessing by index)
    mom_out = id_1_day.loc[id_out,'time']
    mom_in = id_1_day.loc[id_in,'time']


    crit_out_time = re.findall(r'\d+:\d+:\d+',str(mom_out))[0]
    mass_in_time = re.findall(r'\d+:\d+:\d+',str(mom_in))[0]


    #complete time
    critical_outs_times.append(crit_out_time)
    massive_ins_times.append(mass_in_time)
    #Only de hours
    #critical_outs_times.append(int(crit_out_time.split(':')[0]))
    #massive_ins_times.append(int(mass_in_time.split(':')[0]))

print('max_out_time =',critical_outs_times)
print('max_in_time =',massive_ins_times)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


max_out_time= ['01:44:00', '06:45:00', '09:35:00', '21:45:00', '16:20:00', '01:00:00', '08:40:00', '08:50:00', '09:15:00', '22:40:00', '22:50:00', '15:45:00', '10:50:00', '07:45:00', '08:25:00', '16:39:00', '08:24:00', '21:49:00', '21:59:00', '00:49:00', '08:44:00', '08:34:00', '08:24:00', '09:29:00', '08:19:00', '15:39:00', '01:29:00', '09:34:00', '23:04:00', '08:39:00', '08:24:00']
max_in_time= ['04:04:00', '10:35:00', '11:45:00', '11:50:00', '13:55:00', '18:10:00', '09:20:00', '12:40:00', '08:35:00', '07:50:00', '16:05:00', '14:35:00', '21:00:00', '13:25:00', '16:25:00', '14:14:00', '16:39:00', '14:34:00', '02:04:00', '13:19:00', '14:44:00', '15:34:00', '17:29:00', '09:09:00', '11:19:00', '11:34:00', '13:39:00', '09:29:00', '11:14:00', '10:09:00', '09:19:00']


In [319]:
np.mean(critical_outs_times)
np.std(critical_outs_times)
#np.mode(critical_outs_times)

6.563386603307358

# Test with the whole bicing

In [390]:
Ssqlgeneral = 'Select * from st_bicing'
id_st = pd.read_sql(Ssqlgeneral,engine)
id_st.drop('streetName', axis=1, inplace = True)
id_st.drop('streetNumber', axis=1, inplace = True)
id_st.drop('nearbyStations', axis=1, inplace = True)
id_st.drop('updateTime', axis=1, inplace = True)
id_st.drop('month', axis=1, inplace = True)
id_st.head()

,id,type,latitude,longitude,altitude,slots,bikes,status,day,time
0,1,BIKE,41.397952,2.180042,21.0,10,16,OPN,1,00:05:00
1,2,BIKE,41.395530,2.177060,21.0,8,18,OPN,1,00:05:00
2,3,BIKE,41.394055,2.181299,22.0,2,22,OPN,1,00:05:00
3,4,BIKE,41.393480,2.181555,21.0,0,21,OPN,1,00:05:00
4,5,BIKE,41.391075,2.180223,16.0,0,39,OPN,1,00:05:00


# Generating 4 DataFrames with huge amount of data

In [402]:
n_outs = pd.DataFrame()
n_ins = pd.DataFrame()
t_outs = pd.DataFrame()
t_ins = pd.DataFrame()

for station in range(1,241) :
    if station not in (10,43,52,77,194):
        id_st2 = id_st[id_st['id'] == station]    #dataFrame of station 'Station' 

        critical_outs_times = []
        massive_ins_times = []
        critical_outs_list = []
        massive_ins_list = []

        for day in range(1,32):
            id_st_day = id_st2[id_st2['day'] == day]    #dataFrame of day 'day' 
            id_st_day['diff'] = id_st_day['bikes'].diff()

            #looking for the  max/min number of bikes taken in a moment
            critical_outs = id_st_day.min()['diff']
            massive_ins = id_st_day.max()['diff']

            #looking for the index of the rows
            id_out = id_st_day[id_st_day['diff'] == critical_outs].index[0]
            id_in = id_st_day[id_st_day['diff'] == massive_ins].index[0]

            #looking for the MOMENT where this happens (accessing by index)
            mom_out = id_st_day.loc[id_out,'time']
            mom_in = id_st_day.loc[id_in,'time']

            crit_out_time = re.findall(r'\d+:\d+:\d+',str(mom_out))[0]
            mass_in_time = re.findall(r'\d+:\d+:\d+',str(mom_in))[0]


            #appending number of ins/outs to a list
            critical_outs_list.append(critical_outs)
            massive_ins_list.append(massive_ins)

            #appending complete time to a list
            critical_outs_times.append(crit_out_time)
            massive_ins_times.append(mass_in_time)

            #Only de hours (NOT IN USE)
            #critical_outs_times.append(int(crit_out_time.split(':')[0]))
            #massive_ins_times.append(int(mass_in_time.split(':')[0]))

        #print('max_out_time =',critical_outs_times)
        #print('max_in_time =',massive_ins_times)


        #if station%10==0:
        print(station,'station done!')
    
    #fill dataframes (tables)
    n_outs[station] = pd.Series(critical_outs_list)
    n_ins[station] = pd.Series(massive_ins_list)
    t_outs[station] = pd.Series(critical_outs_times)
    t_ins[station] = pd.Series(massive_ins_times)
        

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1 station done!
2 station done!
3 station done!
4 station done!
5 station done!
6 station done!
7 station done!
8 station done!
9 station done!
11 station done!
12 station done!
13 station done!
14 station done!
15 station done!
16 station done!
17 station done!
18 station done!
19 station done!
20 station done!
21 station done!
22 station done!
23 station done!
24 station done!
25 station done!
26 station done!
27 station done!
28 station done!
29 station done!
30 station done!
31 station done!
32 station done!
33 station done!
34 station done!
35 station done!
36 station done!
37 station done!
38 station done!
39 station done!
40 station done!
41 station done!
42 station done!
44 station done!
45 station done!
46 station done!
47 station done!
48 station done!
49 station done!
50 station done!
51 station done!
53 station done!
54 station done!
55 station done!
56 station done!
57 station done!
58 station done!
59 station done!
60 station done!
61 station done!
62 station done!
63 sta

In [412]:
n_outs_trans = n_outs.transpose()
n_ins_trans = n_ins.transpose()
t_outs_trans = t_outs.transpose()
t_ins_trans = t_ins.transpose()

# Functions to treat the data

In [426]:
#Convert dfs to list of lists. Each element of the main list will be a list with the elements of the first row of the df
def df_to_list(df):
        return df.apply(lambda x: x.tolist(), axis=1)

In [527]:
#Function to get usable data. This works with each element of the list of lists i.e. time_list[0],
#time_list[1],..., we will have as many as stations
def usable_data_o(time_list):
    
    summary_total = []
    for stat in time_list:
    
        #We split the data to get a list of hours and a list of minutes
        only_hour = [int(i.split(':', 2)[0]) for i in list(t_outs[stat])]
        only_min = [int(i.split(':', 2)[1]) for i in list(t_outs[stat])]
    
        #Merge the two lists to a list of tuples. Each tuple contains the hour and the minute
        hour_min = list (zip (only_hour, only_min))
    
        #summary_time will be the list we will be able to work with
        summary_time_stat = []
    
        #Summarise the data rounding every time to the closest hour
        for t in hour_min:
            if t[1] < 30:
                summary_time_stat.append(t[0])
            elif t[1] >= 30 and t[1] <60:
                summary_time_stat.append(t[0]+1)
        
        #Append the new list to the final list of lists
        summary_total.append(summary_time_stat)
    
    return(summary_total)

def usable_data_i(time_list):
    
    summary_total = []
    for stat in time_list:
    
        #We split the data to get a list of hours and a list of minutes
        only_hour = [int(i.split(':', 2)[0]) for i in list(t_ins[stat])]
        only_min = [int(i.split(':', 2)[1]) for i in list(t_ins[stat])]
    
        #Merge the two lists to a list of tuples. Each tuple contains the hour and the minute
        hour_min = list (zip (only_hour, only_min))
    
        #summary_time will be the list we will be able to work with
        summary_time_stat = []
    
        #Summarise the data rounding every time to the closest hour
        for t in hour_min:
            if t[1] < 30:
                summary_time_stat.append(t[0])
            elif t[1] >= 30 and t[1] <60:
                summary_time_stat.append(t[0]+1)
        
        #Append the new list to the final list of lists
        summary_total.append(summary_time_stat)
    
    return(summary_total)

In [528]:
#Function to get descriptive indicators
from scipy import stats

def descriptive_stats_o(summary_time,id):
    mean = round(np.mean(summary_time),2)
    median = np.median(summary_time)
    rnge = np.max(summary_time) - np.min(summary_time)
    std = round(np.std(summary_time),2)
    unique_elements, counts_elements = np.unique(summary_time, return_counts=True)
    mode = int(stats.mode(summary_time)[0])
    
    general_o[id] = pd.Series([mean,median,rnge,std,mode], index = ['Mean','Median','Range','Stdev','Mode'])
    
    print(f'For station {id}:\nThe mean is {mean}\nThe median is {median}\nThe mode is {mode}\nThe range is {rnge}\nThe standard deviation is {std}\n---------')

    
#Function to get descriptive indicators


def descriptive_stats_i(summary_time,id):
    mean = round(np.mean(summary_time),2)
    median = np.median(summary_time)
    rnge = np.max(summary_time) - np.min(summary_time)
    std = round(np.std(summary_time),2)
    unique_elements, counts_elements = np.unique(summary_time, return_counts=True)
    mode = int(stats.mode(summary_time)[0])
    
    general_i[id] = pd.Series([mean,median,rnge,std,mode], index = ['Mean','Median','Range','Stdev','Mode'])
    
    print(f'For station {id}:\nThe mean is {mean}\nThe median is {median}\nThe mode is {mode}\nThe range is {rnge}\nThe standard deviation is {std}\n---------')
    

# Statistics for Time DataFrames

In [529]:
# print(np.asarray((unique_elements, counts_elements)))
global general_o 
general_o = pd.DataFrame()

summary_total = usable_data_o(t_outs)
#Main code to get the results
id = 1
for stat in summary_total:
    descriptive_stats_o(stat,id)
    id += 1

For station 1:
The mean is 11.42
The median is 9.0
The mode is 9
The range is 22
The standard deviation is 6.68
---------
For station 2:
The mean is 12.35
The median is 13.0
The mode is 7
The range is 21
The standard deviation is 5.21
---------
For station 3:
The mean is 15.32
The median is 17.0
The mode is 18
The range is 21
The standard deviation is 5.41
---------
For station 4:
The mean is 13.81
The median is 16.0
The mode is 20
The range is 22
The standard deviation is 6.19
---------
For station 5:
The mean is 12.97
The median is 13.0
The mode is 9
The range is 22
The standard deviation is 5.75
---------
For station 6:
The mean is 15.97
The median is 18.0
The mode is 18
The range is 16
The standard deviation is 5.21
---------
For station 7:
The mean is 12.81
The median is 13.0
The mode is 9
The range is 21
The standard deviation is 6.32
---------
For station 8:
The mean is 11.13
The median is 9.0
The mode is 5
The range is 21
The standard deviation is 6.38
---------
For station 9:


For station 181:
The mean is 12.35
The median is 16.0
The mode is 16
The range is 24
The standard deviation is 7.57
---------
For station 182:
The mean is 11.84
The median is 10.0
The mode is 8
The range is 23
The standard deviation is 5.84
---------
For station 183:
The mean is 14.13
The median is 17.0
The mode is 9
The range is 21
The standard deviation is 6.88
---------
For station 184:
The mean is 14.1
The median is 16.0
The mode is 18
The range is 22
The standard deviation is 6.01
---------
For station 185:
The mean is 14.52
The median is 17.0
The mode is 19
The range is 21
The standard deviation is 6.61
---------
For station 186:
The mean is 13.06
The median is 14.0
The mode is 21
The range is 24
The standard deviation is 7.62
---------
For station 187:
The mean is 10.61
The median is 11.0
The mode is 0
The range is 21
The standard deviation is 8.23
---------
For station 188:
The mean is 17.81
The median is 19.0
The mode is 19
The range is 23
The standard deviation is 5.67
------

In [530]:
global general_i
general_i = pd.DataFrame()

summary_total = usable_data_i(t_ins)
#Main code to get the results
id = 1
for stat in summary_total:
    descriptive_stats_i(stat,id)
    id += 1

For station 1:
The mean is 12.42
The median is 13.0
The mode is 9
The range is 19
The standard deviation is 3.95
---------
For station 2:
The mean is 11.81
The median is 13.0
The mode is 18
The range is 18
The standard deviation is 5.59
---------
For station 3:
The mean is 13.58
The median is 14.0
The mode is 12
The range is 21
The standard deviation is 5.15
---------
For station 4:
The mean is 11.45
The median is 13.0
The mode is 14
The range is 19
The standard deviation is 4.65
---------
For station 5:
The mean is 14.32
The median is 16.0
The mode is 16
The range is 19
The standard deviation is 4.71
---------
For station 6:
The mean is 13.13
The median is 13.0
The mode is 12
The range is 20
The standard deviation is 4.52
---------
For station 7:
The mean is 12.23
The median is 12.0
The mode is 8
The range is 22
The standard deviation is 4.27
---------
For station 8:
The mean is 14.16
The median is 14.0
The mode is 14
The range is 23
The standard deviation is 4.91
---------
For statio

For station 194:
The mean is 9.45
The median is 12.0
The mode is 1
The range is 21
The standard deviation is 7.68
---------
For station 195:
The mean is 12.13
The median is 15.0
The mode is 2
The range is 23
The standard deviation is 7.61
---------
For station 196:
The mean is 9.29
The median is 9.0
The mode is 9
The range is 23
The standard deviation is 6.28
---------
For station 197:
The mean is 14.61
The median is 18.0
The mode is 18
The range is 23
The standard deviation is 7.77
---------
For station 198:
The mean is 7.81
The median is 4.0
The mode is 0
The range is 21
The standard deviation is 8.21
---------
For station 199:
The mean is 4.87
The median is 1.0
The mode is 0
The range is 23
The standard deviation is 6.9
---------
For station 200:
The mean is 9.32
The median is 9.0
The mode is 1
The range is 23
The standard deviation is 7.14
---------
For station 201:
The mean is 7.9
The median is 5.0
The mode is 1
The range is 24
The standard deviation is 8.0
---------
For station 2

In [531]:
general_o

,1,2,3,4,5,6,7,8,9,10,...,231,232,233,234,235,236,237,238,239,240
Mean,11.42,12.35,15.32,13.81,12.97,15.97,12.81,11.13,18.16,18.16,...,10.71,17.19,9.23,10.00,17.58,10.97,10.29,12.26,9.52,9.65
Median,9.00,13.00,17.00,16.00,13.00,18.00,13.00,9.00,20.00,20.00,...,9.00,19.00,9.00,9.00,20.00,9.00,9.00,10.00,8.00,9.00
Range,22.00,21.00,21.00,22.00,22.00,16.00,21.00,21.00,22.00,22.00,...,21.00,21.00,24.00,16.00,24.00,21.00,19.00,13.00,19.00,19.00
Stdev,6.68,5.21,5.41,6.19,5.75,5.21,6.32,6.38,5.11,5.11,...,5.08,5.24,5.86,3.71,6.96,5.65,4.52,4.49,3.52,5.80
Mode,9.00,7.00,18.00,20.00,9.00,18.00,9.00,5.00,20.00,20.00,...,8.00,19.00,9.00,9.00,20.00,9.00,9.00,9.00,8.00,9.00


In [532]:
general_i

,1,2,3,4,5,6,7,8,9,10,...,231,232,233,234,235,236,237,238,239,240
Mean,12.42,11.81,13.58,11.45,14.32,13.13,12.23,14.16,16.42,16.42,...,14.39,15.87,8.97,9.23,13.65,10.19,8.84,11.9,10.71,10.74
Median,13.00,13.00,14.00,13.00,16.00,13.00,12.00,14.00,18.00,18.00,...,14.00,17.00,8.00,10.00,17.00,10.00,10.00,12.0,11.00,13.00
Range,19.00,18.00,21.00,19.00,19.00,20.00,22.00,23.00,21.00,21.00,...,21.00,23.00,23.00,18.00,21.00,22.00,15.00,18.0,20.00,21.00
Stdev,3.95,5.59,5.15,4.65,4.71,4.52,4.27,4.91,4.66,4.66,...,5.49,6.36,6.48,3.87,7.90,5.28,3.52,4.8,4.39,7.42
Mode,9.00,18.00,12.00,14.00,16.00,12.00,8.00,14.00,12.00,12.00,...,12.00,8.00,8.00,10.00,20.00,9.00,10.00,14.0,11.00,0.00
